In [12]:
import matplotlib
matplotlib.use('Qt5Agg') 

import matplotlib.pyplot as plt
import cv2 
import numpy as np 
from scipy.spatial.transform import Rotation
from some import add_project_root_to_path

add_project_root_to_path()


K = np.array([[1000,0,300],[0,1000,200],[0,0,1]])
R1,t1 = np.eye(3),np.array([0,0,0])

R2 = Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
t2 = np.array([0.2,2,1])

# Projection matrix for Cam1 and Cam2 

P1 = K @ np.column_stack((R1,t1.reshape(3,1)))
P2 = K @ np.column_stack((R2,t2.reshape(3,1)))


ImportError: Failed to import any of the following Qt binding modules: PyQt5, PySide2

### Exercise 3.1
Consider the 3D point
$Q = [1,0.5,4,1]$

 and find the projections in Cam1 and Cam2, respectively, points q1 and q2

In [ ]:
Q = np.array([1,0.5,4,1]).reshape(4,1)

# Projecting the 3D point to 2D
q1 = P1 @ Q
q2 = P2 @ Q

#Normalizing the coordinates
q1 = q1/q1[2]
q2 = q2/q2[2] 
q1,q2

(array([[550.],
        [325.],
        [  1.]]),
 array([[582.47256835],
        [185.98985776],
        [  1.        ]]))

## Exercise 3.2

Implement a function `CrossOp` that takes a vector in 3D and returns the 3×3 matrix corresponding to taking the cross product with that vector. In the case that $p = \begin{bmatrix} x \\ y \\ z \end{bmatrix}^T$, you should have:

$$
CrossOp(p) = [p]× =
\begin{bmatrix}
0 & -z & y \\
z & 0 & -x \\
-y & x & 0
\end{bmatrix}.
$$

As always, verify that your function works. In this case, you can test it on random vectors and ensure that:

$$
[p1]×p2 = p1 \times p2.
$$

In [ ]:
## Implemented in the functions.py file
def crossOp(p): 
    p_cross = np.array([[0,-p[2],p[1]],[p[2],0,-p[0]],[-p[1],p[0],0]]) 
    return p_cross

### Exercise 3.3
- Compute the fundamental matrix F of the two cameras

The fundamental matrix can be computed from the essential matrix as 

$$ \Large 
F = A_2^{-T} \space  E \space  A_1^{-1}
$$

Where $$\Large E = \space [t]_x \space \space R $$ 

The relative Rotation R from camera 1 to camera 2 can be calculated as  

$$ \Large 
R = R_2 R^T
$$ 

and And the relative translation T can be calculated as 

$$ \Large 
t = t2 - R_2R_1^Tt_1
$$

In [ ]:
R = R2 @ R1.T
t = t2 - R2 @ R1.T @ t1 
E = crossOp(t) @ R 
F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)
print(F)

[[ 3.29311881e-07  8.19396327e-07  1.79162592e-03]
 [ 5.15532551e-07 -8.76915984e-07  9.31426656e-05]
 [-1.29882755e-03  1.51951700e-03 -1.10072682e+00]]


### Exercise 3.4
- What is the epipolar line l of q1 in camera two?

The epipolar line l of q1 in camera two 
is $l = Fq$

In [ ]:
l1 = F @ q1 
l1

array([[ 2.23905126e-03],
       [ 9.16878739e-05],
       [-1.32123895e+00]])

### Exercise 3.5
- Is q2 located on the epipolar line from Exercise 3.4? 
- Do the computations, but also explain why
this must be so.

In [ ]:
# To determine whether the point q2 is located on the epipolar line 
# we can check if the product of q2.T @ l1 is close to zero 

q2.T @ l1


array([[4.4408921e-16]])

## Exercise 3.6

Now assume that both camera one and two have local coordinate systems that are different from the coordinate system of the world.

Let $Q$ and $\tilde{Q}$ denote the same 3D point in world space and in the frame of camera one. In other words, we have the relation:

$$
\tilde{Q} =
\begin{bmatrix}
R_1 & t_1 \\
0 & 1
\end{bmatrix}
Q. \quad (13)
$$

Make sure you understand why this is true.

Show analytically that:

$$
Q =
\begin{bmatrix}
R_1^T & -R_1^T t_1 \\
0 & 1
\end{bmatrix}
\tilde{Q}.
$$


-------------------------------- 
Since the rotation and translation is set to world coordinate system, the pose matrix T maps from homogenous to inhom coordinates. 

$$
Q =
\begin{bmatrix}
R_1^T & -R_1^T t_1 \\
0 & 1
\end{bmatrix}
\tilde{Q}.

\implies Q = \begin{bmatrix}
R_1^T & -R_1^T t_1 \\
0 & 1
\end{bmatrix} 
 \begin{bmatrix}
R_1 & t_1 \\
0 & 1
\end{bmatrix}
Q
$$

$$
 \implies Q = 
\begin{bmatrix}
R_1^T R_1 & R_1^T t_1 \\
0 & 1
\end{bmatrix} Q 

$$

This simplifies to 

$$
 \implies Q = 
\begin{bmatrix}
I & 0 \\
0 & 1
\end{bmatrix} Q 

$$

Since according to Appendix A.4 in the lecture notes (page 198), $R^TR = I$

And the above expression simplifies to Q 

-----------------

### Exercise 3.8

Load the file TwoImageDataCar.npy, and compute the fundamental matrix between the images.



In [ ]:
path = "../Data/" 
Data = np.load(path + "TwoImageDataCar.npy",allow_pickle=True).item()

K = Data['K']
im1 = Data['im1']
R1 = Data['R1']
t1 = Data['t1']
im2 = Data['im2']
R2 = Data['R2']
t2 = Data['t2']


t = t2 - R2 @ R1.T @ t1
R = R2 @ R1.T 

E = crossOp(t) @ R 
F = np.linalg.inv(K).T @ E @ np.linalg.inv(K) 
print(F)

[[-1.50228281e-08 -3.45997421e-07 -3.47606501e-05]
 [-2.06767970e-07  3.96284278e-08 -9.29558240e-04]
 [ 2.61581163e-05  1.12168578e-03  1.17449076e-02]]


________
## Exercise 3.9

Write code that can show both images at the same time. Now write code such that you can click on a point in image one, and display the corresponding epipolar line in image two. Experiment with your code, verifying that the point you click on is on the epipolar line in the other image.

**Tip**: To click on a point and get the pixel coordinates, you can use `plt.ginput(1)`. This needs the figure to open in a new window, so you might need to run `%matplotlib qt` first.

**Tip**: To draw a line given in homogeneous coordinates, you can use the `DrawLine` function below. It takes as input a line in homogeneous coordinates `l` and the size of the image it will be drawn on, as returned by `im.shape`.



In [ ]:
#%matplotlib qt

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2

In [ ]:
from utils.functions_10 import * 

def DrawLine(l, shape):
#Checks where the line intersects the four sides of the image
# and finds the two intersections that are within the frame
    def in_frame(l_im):
        q = np.cross(l.flatten(), l_im)
        q = q[:2]/q[2]
        if all(q>=0) and all(q+1<=shape[1::-1]):
         return q
    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1-shape[1]], [0, 1, 1-shape[0]]]
    P = [in_frame(l_im) for l_im in lines if in_frame(l_im) is not None]
    if (len(P)==0):
        print("Line is completely outside image")
    else:
       print("Line is partially inside image")
       
    plt.plot(*np.array(P).T)







def Show_epipolar_line(im1,im2,K,F):
    fig, ax = plt.subplots(1,2,figsize=(12,6))
    ax[0].imshow(im1)
    ax[1].imshow(im2)
    q1 = np.array(plt.ginput(1,0))
    q1_inh = PiInv(q1.reshape(-1,1)).reshape(1,-1)
    plt.axis('off')
    plt.show(block=False)


    l2 = F @ q1_inh.T

    DrawLine(l2,im2.shape)
    ax[0].scatter(*q1.T, marker='x')
    plt.show()

In [ ]:
Show_epipolar_line(im1,im2,K,F)

Line is partially inside image


----- 

### Exercise 3.10


Do the same thing as the last exercise, but where you can click in image two and get the epipolar
line displayed in image one. You do not compute a new fundamental matrix to do this.

In [ ]:
def DrawLine(l, shape, ax):
    def in_frame(l_im):
        q = np.cross(l.flatten(), l_im)
        q = q[:2] / q[2]
        if all(q >= 0) and all(q + 1 <= shape[1::-1]):
            return q

    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1 - shape[1]], [0, 1, 1 - shape[0]]]
    P = [in_frame(l_im) for l_im in lines if in_frame(l_im) is not None]
    if len(P) == 0:
        print("Line is completely outside image")
    else:
        print("Line is partially inside image")
        # Plot on the specified axes
        ax.plot(*np.array(P).T, color='red')  # You can specify a color for clarity

import matplotlib.pyplot as plt
import numpy as np

def Show_epipolar_line_2(im1, im2, K, F):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(im1)
    ax[1].imshow(im2)
    plt.show(block=False)
    q2 = np.array(plt.ginput(1, timeout=0))  

    q2_inh = PiInv(q2.reshape(-1, 1)).reshape(1, -1)  
    l1 = q2_inh @ F  
    l1 = l1.T
    # Reopen the figure to plot the line
    
    DrawLine(l1.T, im1.shape, ax[0])  # Pass ax[0] to draw the line on the first image
    
    plt.show()



In [ ]:
Show_epipolar_line_2(im1,im2,K,F)

Line is partially inside image


----------
## Exercise 3.11

Create a function named `triangulate`. This function should be capable of triangulating a single 3D point that has been observed by multiple cameras. The function should adhere to the following specifications:

### Function Specifications:
- **Input:**
  - A list of `n` pixel coordinates (q1, q2, ..., qn), representing the observations of the point in each camera's image plane.
  - A list of `n` projection matrices (P1, P2, ..., Pn), where each matrix corresponds to a camera's projection matrix.
- **Output:**
  - The triangulated 3D point using a linear triangulation algorithm.

### Testing the Function:
1. **Define a 3D point**: Choose a point in 3D space.
2. **Project this point onto the image planes of two cameras**: Use the projection matrices of at least two cameras to project the 3D point onto their respective image planes.
3. **Triangulate the 3D point**: Using the projections obtained in the previous step, apply your `triangulate` function to estimate the location of the original 3D point.
4. **Reproject the estimated 3D point back onto the image planes of the cameras**: Check if the reprojected 2D points coincide with the original pixel coordinates used for triangulation.

### Questions:
- After reprojecting your estimated 3D point to the cameras, do you find the same 2D pixels? Discuss any discrepancies or confirmations between the estimated coordinates and the original ones.


In [ ]:
def triangulate(qn,pn):
    """ 
    input:list of points qn and projection matrices Pn"""
    if len(qn) != len(pn):
        raise ValueError("Expected lists of equal length, len(Q)!=len(pn)")



    B_i = lambda P,q: np.array([  [P[2]*q[0] - P[0]],[P[2]*q[1] - P[1]]  ])

    B = np.hstack(([B_i(P_i,Pi(q_i)) for P_i, q_i in zip(pn,qn)]))

    B = B.reshape(len(pn)*2,4)
    U,S,VT = np.linalg.svd(B)

    Q = VT[-1,:]
    return Q




triangulate([q1,q2],[P1,P2])

array([-0.23408229, -0.11704115, -0.93632918, -0.23408229])

In [ ]:
""" Test your function by defining a 3D point, project this point to the image planes of the two
cameras, and then triangulate it using the projection. Try reprojecting your estimated 3D point
to the cameras. Do you find the same 2D pixels?"""

Q = np.array([1,0.5,4,1]).reshape(4,1)
q1 = P1 @ Q 
q2 = P2 @ Q

print((Pi(q1)))
print((Pi(q2)))

[[550.]
 [325.]]
[[582.47256835]
 [185.98985776]]


In [ ]:
Q_est = triangulate([q1,q2],[P1,P2]) 
q1_est = P1 @ Q_est
q2_est = P2 @ Q_est

print((Pi(q1_est)))
print(Pi(q2_est))


[550. 325.]
[582.47256835 185.98985776]


In [ ]:
Q_est

array([ 0.93331152, -0.07474679,  0.26000912, -0.2360885 ])